In [1]:
import random
import numpy as np

# --- Blackjack Game Logic ---

class BlackjackGame:
    def __init__(self):
        self.deck = self.create_deck()
        self.player_hand = []
        self.dealer_hand = []
        self.player_ace = False
        self.dealer_ace = False

    def create_deck(self):
        suits = ['C', 'D', 'H', 'S']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
        deck = [(rank, suit) for suit in suits for rank in ranks]
        random.shuffle(deck)
        return deck

    def deal_card(self, hand):
        if not self.deck:
            self.deck = self.create_deck()  # Reshuffle if deck is empty
        card = self.deck.pop()
        hand.append(card)
        return card

    def get_hand_value(self, hand, has_ace):
        value = 0
        ace_count = 0
        for card in hand:
            rank = card[0]
            if rank.isdigit():
                value += int(rank)
            elif rank in ['T', 'J', 'Q', 'K']:
                value += 10
            elif rank == 'A':
                value += 11
                ace_count += 1

        while value > 21 and ace_count > 0:
            value -= 10
            ace_count -= 1

        return value

    def is_bust(self, hand):
        return self.get_hand_value(hand, self.has_ace(hand)) > 21

    def has_ace(self, hand):
        for card in hand:
            if card[0] == 'A':
                return True
        return False

    def reset_game(self):
        self.deck = self.create_deck()
        self.player_hand = []
        self.dealer_hand = []
        self.player_ace = False
        self.dealer_ace = False

    def play_round(self, player_action):
        self.deal_card(self.player_hand)
        self.deal_card(self.dealer_hand)
        self.deal_card(self.player_hand)
        self.deal_card(self.dealer_hand)

        self.player_ace = self.has_ace(self.player_hand)
        self.dealer_ace = self.has_ace(self.dealer_hand)

        if self.get_hand_value(self.player_hand, self.player_ace) == 21 and self.get_hand_value(self.dealer_hand, self.dealer_ace) == 21:
            return 0  # Push

        if self.get_hand_value(self.player_hand, self.player_ace) == 21:
            return 1  # Player Blackjack

        if self.get_hand_value(self.dealer_hand, self.dealer_ace) == 21:
            return -1 # Dealer Blackjack

        if self.is_bust(self.player_hand):
            return -1 # Player Bust

        if self.is_bust(self.dealer_hand):
            return 1  # Dealer Bust

        while self.get_hand_value(self.dealer_hand, self.dealer_ace) < 17:
            self.deal_card(self.dealer_hand)
            self.dealer_ace = self.has_ace(self.dealer_hand)
            if self.is_bust(self.dealer_hand):
                return 1

        player_value = self.get_hand_value(self.player_hand, self.player_ace)
        dealer_value = self.get_hand_value(self.dealer_hand, self.dealer_ace)

        if player_value > dealer_value:
            return 1
        elif dealer_value > player_value:
            return -1
        else:
            return 0

# --- Monte Carlo Control ---

class MonteCarloBlackjack:
    def __init__(self, num_episodes=10000, learning_rate=0.1, epsilon=0.1):
        self.num_episodes = num_episodes
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.Q = {}  # Q-value table (state, action) -> value
        self.policy = {} # Policy (state) -> action
        self.game = BlackjackGame()

    def get_state(self):
        player_sum = self.game.get_hand_value(self.game.player_hand, self.game.player_ace)
        dealer_up_card = self.game.dealer_hand[0][0] if self.game.dealer_hand else None
        has_usable_ace = self.game.player_ace and player_sum <= 11
        dealer_showing_value = 0
        if dealer_up_card:
            if dealer_up_card.isdigit():
                dealer_showing_value = int(dealer_up_card)
            elif dealer_up_card in ['T', 'J', 'Q', 'K']:
                dealer_showing_value = 10
            elif dealer_up_card == 'A':
                dealer_showing_value = 11

        return (player_sum, dealer_showing_value, has_usable_ace)

    def get_actions(self):
        return ["hit", "stick"]

    def get_initial_policy(self):
        policy = {}
        for sum_val in range(4, 22):
            for dealer_val in range(2, 12):
                for has_ace in [False, True]:
                    state = (sum_val, dealer_val, has_ace)
                    if sum_val < 17:
                        policy[state] = "hit"
                    else:
                        policy[state] = "stick"
        return policy

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.get_actions()) # Explore
        else:
            return self.policy.get(state, "stick") # Exploit (default to stick if state not in policy)

    def generate_episode(self, current_policy):
        episode = []
        self.game.reset_game()
        state = self.get_state()
        while True:
            if state[0] > 21:
                reward = -1
                episode.append((state, "bust", reward))
                break

            action = current_policy.get(state, "stick") # Use current policy
            episode.append((state, action, None)) # Reward will be assigned at the end

            if action == "stick":
                reward = self.game.play_round("stick")
                episode = [(s, a, reward) for s, a, _ in episode] # Assign reward to all steps
                break
            elif action == "hit":
                self.game.deal_card(self.game.player_hand)
                self.game.player_ace = self.game.has_ace(self.game.player_hand)
                next_state = self.get_state()
                if next_state[0] > 21:
                    reward = -1
                    episode = [(s, a, reward) for s, a, _ in episode]
                    break
                state = next_state
        return episode

    def update_q_values(self, episode):
        states_actions_visited = set()
        returns = {}
        G = 0
        for i in reversed(range(len(episode))):
            state, action, reward = episode[i]
            G = reward if reward is not None else 0 # No discounting in Blackjack

            state_action = (state, action)
            if state_action not in states_actions_visited:
                states_actions_visited.add(state_action)
                if state_action not in returns:
                    returns[state_action] = []
                returns[state_action].append(G)

        for (state, action), returns_list in returns.items():
            if (state, action) not in self.Q:
                self.Q[(state, action)] = 0
            self.Q[(state, action)] += self.learning_rate * (np.mean(returns_list) - self.Q[(state, action)])

    def improve_policy(self):
        policy_updated = False
        for sum_val in range(4, 22):
            for dealer_val in range(2, 12):
                for has_ace in [False, True]:
                    state = (sum_val, dealer_val, has_ace)
                    best_action = None
                    max_q = -float('inf')
                    for action in self.get_actions():
                        q_value = self.Q.get((state, action), 0)
                        if q_value > max_q:
                            max_q = q_value
                            best_action = action
                        elif q_value == max_q:
                            # Tie-breaking (optional, can be random)
                            if random.random() < 0.5:
                                best_action = action

                    if state not in self.policy or self.policy[state] != best_action:
                        self.policy[state] = best_action
                        policy_updated = True
        return policy_updated

    def train(self):
        self.policy = self.get_initial_policy()
        for episode_num in range(self.num_episodes):
            episode = self.generate_episode(self.policy)
            self.update_q_values(episode)
            policy_updated = self.improve_policy()
            if episode_num % 1000 == 0:
                print(f"Episode {episode_num}, Policy Updated: {policy_updated}")
                # You can add evaluation of the policy here if needed

        print("Training finished.")
        # Print a sample of the learned policy
        print("\nLearned Policy (Sample):")
        for state, action in list(self.policy.items())[:10]:
            print(f"State: {state}, Action: {action}")

# --- Run the Training ---

if __name__ == "__main__":
    mc_blackjack = MonteCarloBlackjack(num_episodes=50000) # Increase episodes for better learning
    mc_blackjack.train()

Episode 0, Policy Updated: True
Episode 1000, Policy Updated: True
Episode 2000, Policy Updated: True
Episode 3000, Policy Updated: True
Episode 4000, Policy Updated: True
Episode 5000, Policy Updated: True
Episode 6000, Policy Updated: True
Episode 7000, Policy Updated: True
Episode 8000, Policy Updated: True
Episode 9000, Policy Updated: True
Episode 10000, Policy Updated: True
Episode 11000, Policy Updated: True
Episode 12000, Policy Updated: True
Episode 13000, Policy Updated: True
Episode 14000, Policy Updated: True
Episode 15000, Policy Updated: True
Episode 16000, Policy Updated: True
Episode 17000, Policy Updated: True
Episode 18000, Policy Updated: True
Episode 19000, Policy Updated: True
Episode 20000, Policy Updated: True
Episode 21000, Policy Updated: True
Episode 22000, Policy Updated: True
Episode 23000, Policy Updated: True
Episode 24000, Policy Updated: True
Episode 25000, Policy Updated: True
Episode 26000, Policy Updated: True
Episode 27000, Policy Updated: True
Episo